In [1]:
getwd()
if ( is.null(environment()$this_notebook_dir) ) {
    this_notebook_dir <- getwd()
    setwd(paste0(getwd(), '/..'))
}
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/NA_fix"

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

In [3]:
source('helpers.R')

kaggle.house.loadLibraries()
data <- kaggle.house.loadData()

data$combined %>% 
select(-dataSource) %>% 
filter(!is.na(LotFrontage)) %>% 
mutate(
    LotFrontage.log = log(LotFrontage),
    LotArea.log = log(LotArea)
) -> df.LotFrontage

df.LotFrontage %>% dim

[1] 2433   81

In [4]:
df <- df.LotFrontage %>%
select(LotFrontage.log, LotArea.log, MSSubClass, Neighborhood) %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass + Neighborhood, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)


Call:
lm(formula = LotFrontage.log ~ LotArea.log + MSSubClass + Neighborhood, 
    data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.94486 -0.08362 -0.00344  0.09205  1.12980 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)          1.33677    0.11911  11.223  < 2e-16 ***
LotArea.log          0.30096    0.01377  21.849  < 2e-16 ***
MSSubClass160       -0.16048    0.03153  -5.089 3.88e-07 ***
MSSubClass180       -0.15344    0.05955  -2.577 0.010038 *  
MSSubClass190        0.35306    0.03649   9.676  < 2e-16 ***
MSSubClass20         0.39720    0.02245  17.696  < 2e-16 ***
MSSubClass30         0.32701    0.03061  10.683  < 2e-16 ***
MSSubClass40         0.16832    0.09232   1.823 0.068401 .  
MSSubClass45         0.26417    0.05259   5.023 5.46e-07 ***
MSSubClass50         0.31108    0.02731  11.390  < 2e-16 ***
MSSubClass60         0.39258    0.02355  16.671  < 2e-16 ***
MSSubClass70         0.31639    0.03152  10.039  < 2e-1

[1] 94.15694